In [1]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    GROQ_API_KEY = "your_api_key_here",
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
pip install --upgrade langchain langchain-community


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 3.1 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.3 MB/s  0:00:00

   ----- ----------------------------------  2/14 [multidict]
   -------- -------------------------------  3/14 [marshmallow]
   -------- -------------------------------  3/14 [marshmallow]
   ----------- ----------------------------  4/14 [httpx-sse]
   ----------------- ----------------------  6/14 [aiohappyeyeballs]
   -------------------- -------------------  7/14 [yarl]
   ---------------------- -----------------  8/14 [typing-inspect]
   ---------------------------- ----------- 10/14 [pydantic-settings]
   ---------------------------- ----------- 10/14 [pydantic-settings]
   ---------------------------- ----------- 10/14 [pydantic-settings]
   ---------------------------- ----------- 10/14 

In [9]:
import os
os.environ["USER_AGENT"] = "MyPythonApp/1.0"
from langchain_community.document_loaders import WebBaseLoader


In [12]:
#from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-iii-itc/job/R-65548")
page_data = loader.load().pop().page_content
clean_text = "\n".join(line.strip() for line in page_data.splitlines() if line.strip())

print(clean_text)

Software Engineer III, ITC
Skip to main content
Open Virtual Assistant
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Language
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Careers
Close Menu
Careers
Chat
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Jordan Careers
Converse Careers
Language
Menu
Return to Previous Menu
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Back to Search
Software Engineer III, ITC
Categories ID
Categories URL
Position Type
Full Time
Date Posted
Primary Quest ID
Second Quest ID
Job Classification
Career area
Software Engineering
Location
4/F, WeWork, Embassy GolfLinks, Karnataka, Karnataka 560093, India
Job ID
R-65548
Apply Now
Share Job
Share Job Posting
Facebook
Opens In A New Tab
LinkedIn
Link Opens In New Window
Email
Close-Medium (Def

In [13]:
from langchain_core.prompts import PromptTemplate

In [15]:
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III, ITC',
 'experience': '5-9 years',
 'skills': ['AWS Cloud Services',
  'Kubernetes',
  'DevOps',
  'Terraform',
  'Node JS',
  'Go',
  "API's",
  'PaaS architecture',
  'Jenkins',
  'Docker',
  'Gen AI Tools'],
 'description': 'We are looking for an ambitious Software Engineer III– Platforms with a passion for cloud-native development and platform ownership. You are someone who thrives in a collaborative environment, is excited by cutting-edge technology, and excels at problem-solving.'}

In [17]:
type(json_res)

dict

In [18]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [19]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
links = collection.query(query_texts=['Experience in Python','Experience in react'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [24]:
job

{'role': 'Software Engineer III, ITC',
 'experience': '5-9 years',
 'skills': ['AWS Cloud Services',
  'Kubernetes',
  'DevOps',
  'Terraform',
  'Node JS',
  'Go',
  "API's",
  'PaaS architecture',
  'Jenkins',
  'Docker',
  'Gen AI Tools'],
 'description': 'We are looking for an ambitious Software Engineer III– Platforms with a passion for cloud-native development and platform ownership. You are someone who thrives in a collaborative environment, is excited by cutting-edge technology, and excels at problem-solving.'}

In [22]:
job = json_res
job['skills']

['AWS Cloud Services',
 'Kubernetes',
 'DevOps',
 'Terraform',
 'Node JS',
 'Go',
 "API's",
 'PaaS architecture',
 'Jenkins',
 'Docker',
 'Gen AI Tools']

In [23]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfol

In [25]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Priya, a solutions consultant at NovaTech Solutions. NovaTech is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of NovaTech 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase NovaTech's portfolio: {link_list}
    Remember you are Priya, Solutions Consultant at NovaTech. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your ITC Needs

Dear Hiring Manager,

I came across the job description for a Software Engineer III, ITC, and I am excited to introduce NovaTech Solutions as a potential partner in fulfilling your requirements. With our expertise in cloud-native development, platform ownership, and cutting-edge technology, we are confident that our team can meet your needs.

At NovaTech, we have a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in AWS Cloud Services, Kubernetes, DevOps, Terraform, Node JS, Go, API's, PaaS architecture, Jenkins, and Docker, making us an ideal fit for your project.

I would like to highlight some of our relevant portfolio links that demonstrate our capabilities:

* https://example.com/devops-portfolio (showcasing our expertise in DevOps)
* https://example.com/